In [1]:
#importing libs
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [2]:
#Importing classes for Feature Engineering and Random Forest classifcation
from sklearn.model_selection import train_test_split
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import LabelEncoder, OneHotEncoder
from sklearn.impute import SimpleImputer
from sklearn.ensemble import RandomForestClassifier

In [3]:
#Reading Train data
dataset = pd.read_csv('train.csv')

In [4]:
dataset.head()

,pet_id,issue_date,listing_date,condition,color_type,length(m),height(cm),X1,X2,breed_category,pet_category
0,ANSL_69903,2016-07-10 00:00:00,2016-09-21 16:25:00,2.0,Brown Tabby,0.80,7.78,13,9,0.0,1
1,ANSL_66892,2013-11-21 00:00:00,2018-12-27 17:47:00,1.0,White,0.72,14.19,13,9,0.0,2
2,ANSL_69750,2014-09-28 00:00:00,2016-10-19 08:24:00,NaN,Brown,0.15,40.90,15,4,2.0,4
3,ANSL_71623,2016-12-31 00:00:00,2019-01-25 18:30:00,1.0,White,0.62,17.82,0,1,0.0,2
4,ANSL_57969,2017-09-28 00:00:00,2017-11-19 09:38:00,2.0,Black,0.50,11.06,18,4,0.0,1


In [5]:
#Seperating target and dependent variabel
Y = dataset[['breed_category','pet_category']].values
X = dataset.drop(labels=['pet_id','issue_date','listing_date','breed_category','pet_category'], axis = 1).values

In [6]:
#Spliting the data set into train and validation set
X_train, X_test, Y_train, Y_test =  train_test_split(X, Y, test_size=0.3, random_state=0)

In [7]:
print(X.shape)
print(X_train.shape)
print(X_test.shape)
print(Y_train.shape)
print(Y_test.shape)

(18834, 6)
(13183, 6)
(5651, 6)
(13183, 2)
(5651, 2)


In [8]:
#Remvoing null value from condition column
imputer = SimpleImputer(missing_values=np.nan, strategy='mean')
imputer = imputer.fit(X_train[:,0].reshape(-1, 1))
Z = imputer.transform(X_train[:,0].reshape(-1, 1))
Z_test = imputer.transform(X_test[:,0].reshape(-1,1))

In [9]:
X_train[:,0]=Z.flatten() 
X_test[:,0]=Z_test.flatten() 

In [10]:
print(X_train.shape)
print(X_test.shape)

(13183, 6)
(5651, 6)


In [11]:
print(X_train)

[[2.0 'Orange Tabby' 0.87 27.85 13 9]
 [1.0 'Blue' 0.07 5.34 0 1]
 [0.0 'Brown' 0.75 35.05 13 9]
 ...
 [2.0 'Blue Tabby' 0.94 31.45 13 9]
 [0.0 'Brown' 0.08 35.33 7 1]
 [0.8902047865778435 'Orange Tabby' 0.4 19.77 0 4]]


In [12]:
#Doing categorical encodeing for color column
ct = ColumnTransformer(transformers = [('encoder',OneHotEncoder(), [1])], remainder= 'passthrough')
X_train = ct.fit_transform(X_train)

In [13]:
X_test = ct.transform(X_test)

In [14]:
#Creating random forest object
classifier = RandomForestClassifier(n_estimators=6, criterion='entropy', random_state=0)

In [15]:
classifier.fit(X_train, Y_train)

RandomForestClassifier(bootstrap=True, ccp_alpha=0.0, class_weight=None,
                       criterion='entropy', max_depth=None, max_features='auto',
                       max_leaf_nodes=None, max_samples=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, n_estimators=6,
                       n_jobs=None, oob_score=False, random_state=0, verbose=0,
                       warm_start=False)

In [16]:
#Predicting the result
Y_predr = classifier.predict(X_test)

In [18]:
Y_testl = Y_test.tolist()
Y_predl1 = Y_predr.tolist()
len(Y_testl)

5651

In [19]:
count = 0
for i in range(5641):
    if Y_testl[i] == Y_predl1[i]:
        count+=1
count

4239

In [20]:
#Reading Test data from here
test=pd.read_csv('test.csv')

In [21]:
test.head()

,pet_id,issue_date,listing_date,condition,color_type,length(m),height(cm),X1,X2
0,ANSL_75005,2005-08-17 00:00:00,2017-09-07 15:35:00,0.0,Black,0.87,42.73,0,7
1,ANSL_76663,2018-11-15 00:00:00,2019-05-08 17:24:00,1.0,Orange Tabby,0.06,6.71,0,1
2,ANSL_58259,2012-10-11 00:00:00,2018-04-02 16:51:00,1.0,Black,0.24,41.21,0,7
3,ANSL_67171,2015-02-13 00:00:00,2018-04-06 07:25:00,1.0,Black,0.29,8.46,7,1
4,ANSL_72871,2017-01-18 00:00:00,2018-04-26 13:42:00,1.0,Brown,0.71,30.92,0,7


In [22]:
T = test.drop(labels=['pet_id','issue_date','listing_date'], axis = 1).values

In [23]:
imputer_test = imputer.transform(T[:,0].reshape(-1,1))

In [24]:
T[:,0]=imputer_test.flatten() 

In [25]:
T_test = ct.transform(T)

In [26]:
#Prediting train result
T_pred = classifier.predict(T_test)

In [27]:
range(len(T_pred))

range(0, 8072)

In [28]:
#writing to the CSV feed
f = open('result.csv',"w")
count=0
f.write('pet_id,breed_category,pet_category')
for i in range(len(T_pred)):
    f.write(f'\n{test["pet_id"][i]},{int(T_pred[i][0])},{int(T_pred[i][1])} ')
    count+=1
f.close()